In [1]:
import pandas as pd
import numpy as np
import os
import math
import re
from collections import Counter
import random
import scipy.stats
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [2]:
path11 = r'E:\netData\MAGzip\MAG Nobel\2 Nobelpapers\712 NobelpapersCD.zip'
path12 = r'E:\netData\MAGzip\MAG Nobel\2 Nobelpapers\712 Nobelpapers.zip'
path21 = r'E:\netData\MAGzip\MAG Nobel\2 NobelpaperControls\20954 NobelpapercontrolsCD.zip'
path22 = r'E:\netData\MAGzip\MAG Nobel\2 NobelpaperControls\20954 Nobelpapercontrols.zip'

In [3]:
fNobel1 = pd.read_csv(path11)
fControl1 = pd.read_csv(path21)
fNobel2 = pd.read_csv(path12)
fControl2 = pd.read_csv(path22)

In [4]:
fNobel = pd.concat([fNobel1 , fNobel2] , axis = 1)
fControl = pd.concat([fControl1 , fControl2] , axis = 1)

fNobel = fNobel.loc[:, ~fNobel.columns.duplicated()]
fControl = fControl.loc[:, ~fControl.columns.duplicated()]

In [5]:
fNobel['type'] = 1
fControl['type'] = 0
df = pd.concat([fNobel , fControl])

In [6]:
df['paperlength'] = df['lastpage'] - df['1stpage']

In [7]:
df['journalid'] = df['journal'].map(dict(zip(df['journal'].unique() , range(len(df['journal'].unique())))))

In [8]:
data = df[['type','CD','ni','ref','cit','AUnum','paperlength','paperyear','journalid']]

In [9]:
data = data.fillna(0)

In [10]:
def save_output(model , savepath):
    params = model.params
    bse = model.bse
    pvalues = model.pvalues
    # create a DataFrame with the results
    results_df = pd.DataFrame({'coef': params,'std err': bse,'p_value': pvalues})
    results_df = results_df.round(4)
    # add asterisks to the p-values based on the significance level
    results_df['p_value'] = results_df['p_value'].apply(lambda x: '{:.3f}'.format(x))
    results_df['p_value'] = results_df['p_value'].astype(float)
    results_df['p_star'] = ''
    results_df.loc[results_df['p_value'] < 0.01, 'p_star'] = '***'
    results_df.loc[(results_df['p_value'] >= 0.01) & (results_df['p_value'] < 0.05), 'p_star'] = '**'
    results_df.loc[(results_df['p_value'] >= 0.05) & (results_df['p_value'] < 0.1), 'p_star'] = '*'
    results_df['p_value'] = results_df['p_value'].apply(lambda x: '{:.3f}'.format(x))
    results_df['v'] = results_df['coef'].astype(str) + results_df['p_star'] +'\n'+ '('+ results_df['std err'].astype(str) +')'
    results_df.to_excel(savepath)

# Dependent variable: ni (DC) OLS

In [38]:
formula = 'ni ~  type + cit + ref + AUnum + paperlength + C(journalid) + C(paperyear)'
model = sm.formula.ols(formula=formula, data=data).fit()
print(model.summary())
save_output(model , 'E:/00.xlsx')

                            OLS Regression Results                            
Dep. Variable:                     ni   R-squared:                       0.913
Model:                            OLS   Adj. R-squared:                  0.912
Method:                 Least Squares   F-statistic:                     887.0
Date:                Mon, 22 May 2023   Prob (F-statistic):               0.00
Time:                        13:19:39   Log-Likelihood:            -1.4731e+05
No. Observations:               21666   AIC:                         2.951e+05
Df Residuals:                   21411   BIC:                         2.972e+05
Df Model:                         254                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 61

In [39]:
formula = 'ni ~  type + ref + AUnum + paperlength + C(journalid) + C(paperyear)'
model = sm.formula.ols(formula=formula, data=data).fit()
print(model.summary())
save_output(model , 'E:/01.xlsx')

                            OLS Regression Results                            
Dep. Variable:                     ni   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.110
Method:                 Least Squares   F-statistic:                     11.54
Date:                Mon, 22 May 2023   Prob (F-statistic):               0.00
Time:                        13:19:39   Log-Likelihood:            -1.7240e+05
No. Observations:               21666   AIC:                         3.453e+05
Df Residuals:                   21412   BIC:                         3.473e+05
Df Model:                         253                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept               -175

In [40]:
formula = 'ni ~  type + C(journalid) + C(paperyear)'
model = sm.formula.ols(formula=formula, data=data).fit()
print(model.summary())
save_output(model , 'E:/02.xlsx')

                            OLS Regression Results                            
Dep. Variable:                     ni   R-squared:                       0.118
Model:                            OLS   Adj. R-squared:                  0.108
Method:                 Least Squares   F-statistic:                     11.51
Date:                Mon, 22 May 2023   Prob (F-statistic):               0.00
Time:                        13:19:40   Log-Likelihood:            -1.7242e+05
No. Observations:               21666   AIC:                         3.453e+05
Df Residuals:                   21415   BIC:                         3.473e+05
Df Model:                         250                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept               -125

In [41]:
formula = 'ni ~  type + cit + ref + AUnum + paperlength'
model = sm.formula.ols(formula=formula, data=data).fit()
print(model.summary())
save_output(model , 'E:/04.xlsx')

                            OLS Regression Results                            
Dep. Variable:                     ni   R-squared:                       0.912
Model:                            OLS   Adj. R-squared:                  0.912
Method:                 Least Squares   F-statistic:                 4.473e+04
Date:                Mon, 22 May 2023   Prob (F-statistic):               0.00
Time:                        13:19:40   Log-Likelihood:            -1.4750e+05
No. Observations:               21666   AIC:                         2.950e+05
Df Residuals:                   21660   BIC:                         2.951e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      12.5237      1.740      7.196      

# Dependent variable: ni (DC) NegativeBinomial

In [29]:
formula = 'ni ~  type + cit + ref + AUnum + paperlength + C(journalid) + C(paperyear)'
model = sm.GLM.from_formula(formula, data=data, family=sm.families.NegativeBinomial()).fit()
print(model.summary())
save_output(model , 'E:/10.xlsx')

                 Generalized Linear Model Regression Results                  
Dep. Variable:                     ni   No. Observations:                21666
Model:                            GLM   Df Residuals:                    21411
Model Family:        NegativeBinomial   Df Model:                          254
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -90688.
Date:                Mon, 22 May 2023   Deviance:                   4.5562e+07
Time:                        11:44:27   Pearson chi2:                 3.27e+04
No. Iterations:                   100   Pseudo R-squ. (CS):             0.9148
Covariance Type:            nonrobust                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  2

In [30]:
formula = 'ni ~  type + ref + AUnum + paperlength + C(journalid) + C(paperyear)'
model = sm.GLM.from_formula(formula, data=data, family=sm.families.NegativeBinomial()).fit()
print(model.summary())
save_output(model , 'E:/11.xlsx')

                 Generalized Linear Model Regression Results                  
Dep. Variable:                     ni   No. Observations:                21666
Model:                            GLM   Df Residuals:                    21412
Model Family:        NegativeBinomial   Df Model:                          253
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -97355.
Date:                Mon, 22 May 2023   Deviance:                       50205.
Time:                        11:44:53   Pearson chi2:                 1.84e+05
No. Iterations:                   100   Pseudo R-squ. (CS):             0.8424
Covariance Type:            nonrobust                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  1

In [31]:
formula = 'ni ~  type + C(journalid) + C(paperyear)'
model = sm.GLM.from_formula(formula, data=data, family=sm.families.NegativeBinomial()).fit()
print(model.summary())
save_output(model , 'E:/12.xlsx')

                 Generalized Linear Model Regression Results                  
Dep. Variable:                     ni   No. Observations:                21666
Model:                            GLM   Df Residuals:                    21415
Model Family:        NegativeBinomial   Df Model:                          250
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -98445.
Date:                Mon, 22 May 2023   Deviance:                       52384.
Time:                        11:45:14   Pearson chi2:                 1.77e+05
No. Iterations:                    16   Pseudo R-squ. (CS):             0.8257
Covariance Type:            nonrobust                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  2

In [32]:
formula = 'ni ~  type + cit + ref + AUnum + paperlength'
model = sm.GLM.from_formula(formula, data=data, family=sm.families.NegativeBinomial()).fit()
print(model.summary())
save_output(model , 'E:/13.xlsx')

                 Generalized Linear Model Regression Results                  
Dep. Variable:                     ni   No. Observations:                21666
Model:                            GLM   Df Residuals:                    21660
Model Family:        NegativeBinomial   Df Model:                            5
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -93727.
Date:                Mon, 22 May 2023   Deviance:                   5.5221e+07
Time:                        11:45:15   Pearson chi2:                 2.95e+04
No. Iterations:                   100   Pseudo R-squ. (CS):             0.8872
Covariance Type:            nonrobust                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       2.7282      0.008    335.048      

# Dependent variable: CD

In [33]:
formula = 'CD ~  type + cit + ref + AUnum + paperlength + C(journalid) + C(paperyear)'
model = sm.formula.ols(formula=formula, data=df).fit()
print(model.summary())
save_output(model , 'E:/20.xlsx')

                            OLS Regression Results                            
Dep. Variable:                     CD   R-squared:                       0.553
Model:                            OLS   Adj. R-squared:                  0.547
Method:                 Least Squares   F-statistic:                     104.3
Date:                Mon, 22 May 2023   Prob (F-statistic):               0.00
Time:                        11:55:20   Log-Likelihood:                -6330.3
No. Observations:               21091   AIC:                         1.316e+04
Df Residuals:                   20843   BIC:                         1.513e+04
Df Model:                         247                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  0

In [34]:
formula = 'CD ~  type + ref + AUnum + paperlength + C(journalid) + C(paperyear)'
model = sm.formula.ols(formula=formula, data=df).fit()
print(model.summary())
save_output(model , 'E:/21.xlsx')

                            OLS Regression Results                            
Dep. Variable:                     CD   R-squared:                       0.552
Model:                            OLS   Adj. R-squared:                  0.547
Method:                 Least Squares   F-statistic:                     104.6
Date:                Mon, 22 May 2023   Prob (F-statistic):               0.00
Time:                        11:57:34   Log-Likelihood:                -6338.7
No. Observations:               21091   AIC:                         1.317e+04
Df Residuals:                   20844   BIC:                         1.514e+04
Df Model:                         246                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  0

In [35]:
formula = 'CD ~  type + C(journalid) + C(paperyear)'
model = sm.formula.ols(formula=formula, data=df).fit()
print(model.summary())
save_output(model , 'E:/22.xlsx')

                            OLS Regression Results                            
Dep. Variable:                     CD   R-squared:                       0.475
Model:                            OLS   Adj. R-squared:                  0.469
Method:                 Least Squares   F-statistic:                     77.65
Date:                Mon, 22 May 2023   Prob (F-statistic):               0.00
Time:                        11:57:34   Log-Likelihood:                -8233.1
No. Observations:               21666   AIC:                         1.697e+04
Df Residuals:                   21415   BIC:                         1.897e+04
Df Model:                         250                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  0

In [36]:
formula = 'CD ~  type + cit + ref + AUnum + paperlength'
model = sm.formula.ols(formula=formula, data=df).fit()
print(model.summary())
save_output(model , 'E:/23.xlsx')

                            OLS Regression Results                            
Dep. Variable:                     CD   R-squared:                       0.265
Model:                            OLS   Adj. R-squared:                  0.265
Method:                 Least Squares   F-statistic:                     1522.
Date:                Mon, 22 May 2023   Prob (F-statistic):               0.00
Time:                        11:57:34   Log-Likelihood:                -11566.
No. Observations:               21091   AIC:                         2.314e+04
Df Residuals:                   21085   BIC:                         2.319e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.6120      0.003    179.468      